In [12]:
import pandas as pd 



In [2]:
data = pd.read_csv("/Users/prathameshankaram/Desktop/Project2.csv")

In [3]:
data.head(), data.columns

(     Date        XOM       DRI   MKT    RF
 0  1/3/05   4.375902 -0.177035  5.57  0.12
 1  2/1/05 -13.241701 -3.201101 -3.65  0.11
 2  3/1/05  -1.491942 -7.527938 -2.35  0.11
 3  4/1/05   4.208550  8.926722  0.28  0.14
 4  5/2/05   4.488743 -5.181345  2.65  0.14,
 Index(['Date', 'XOM', 'DRI', 'MKT', 'RF'], dtype='object'))

In [4]:
# Calculating the average return and standard deviation for Exxon Mobil (XOM) and Darden Restaurants (DRI)
average_xom = data['XOM'].mean()
std_dev_xom = data['XOM'].std()
average_dri = data['DRI'].mean()
std_dev_dri = data['DRI'].std()

# Calculating the correlation between XOM and DRI
correlation_xom_dri = data['XOM'].corr(data['DRI'])

print(average_xom), (std_dev_xom), (average_dri), (std_dev_dri), (correlation_xom_dri)


0.9543722582833337


(None,
 10.049779345834818,
 1.422620666283333,
 10.630195656869258,
 0.5193835641540854)

In [5]:
import numpy as np

# Convert annual target standard deviation to monthly
annual_target_std_dev = 15
monthly_target_std_dev = annual_target_std_dev / np.sqrt(12) / 100  # convert percentage to decimal

# Average monthly risk-free rate
average_rf = data['RF'].mean() / 100  # convert percentage to decimal

# Calculate the weight of the stock in the portfolio to meet the target std deviation
weight_xom = monthly_target_std_dev / std_dev_xom
weight_dri = monthly_target_std_dev / std_dev_dri

# Calculate the expected return of the portfolio
expected_return_xom = weight_xom * average_xom + (1 - weight_xom) * average_rf
expected_return_dri = weight_dri * average_dri + (1 - weight_dri) * average_rf

# Calculate the weight of the risk-free asset in each portfolio
weight_rf_xom = 1 - weight_xom
weight_rf_dri = 1 - weight_dri

print(monthly_target_std_dev), (expected_return_xom), (weight_rf_xom), (expected_return_dri), (weight_rf_dri)


0.04330127018922194


(None,
 0.005079563486636896,
 0.9956913213017788,
 0.006762642621738448,
 0.9959265782505856)

In [6]:
# Define the weights for Exxon and Darden in the portfolio
weight_xom = 0.8
weight_dri = 0.2

# Calculate the average return of the portfolio
portfolio_avg_return = (weight_xom * average_xom) + (weight_dri * average_dri)

# Calculate the standard deviation of the portfolio
portfolio_std_dev = np.sqrt(
    (weight_xom ** 2 * std_dev_xom ** 2) +
    (weight_dri ** 2 * std_dev_dri ** 2) +
    (2 * weight_xom * weight_dri * std_dev_xom * std_dev_dri * correlation_xom_dri)
)

# Calculate the Sharpe Ratio of the portfolio
sharpe_ratio_portfolio = (portfolio_avg_return - average_rf) / portfolio_std_dev

print(portfolio_avg_return), (portfolio_std_dev), (sharpe_ratio_portfolio)


1.0480219398833337


(None, 9.322790835760626, 0.11231081890203541)

In [7]:
# Calculate the excess returns for XOM, DRI, and the Market
data['Excess_XOM'] = data['XOM'] - data['RF']
data['Excess_DRI'] = data['DRI'] - data['RF']
data['Excess_MKT'] = data['MKT'] - data['RF']

# Calculate the covariance of XOM and DRI with the Market
cov_xom_mkt = data['Excess_XOM'].cov(data['Excess_MKT'])
cov_dri_mkt = data['Excess_DRI'].cov(data['Excess_MKT'])

# Calculate the variance of the Market
var_mkt = data['Excess_MKT'].var()

# Calculate beta for XOM and DRI
beta_xom = cov_xom_mkt / var_mkt
beta_dri = cov_dri_mkt / var_mkt

# Calculate the expected returns according to CAPM
capm_expected_return_xom = average_rf + beta_xom * (data['Excess_MKT'].mean())
capm_expected_return_dri = average_rf + beta_dri * (data['Excess_MKT'].mean())

print(beta_xom), (capm_expected_return_xom), (beta_dri), (capm_expected_return_dri)


1.0603088461430916


(None, 0.6949438064673202, 1.1888071174057981, 0.7790459250087617)

In [8]:
# Updated correlation
correlation_xom_dri_updated = 0.6

# Calculate the standard deviation of the portfolio with updated correlation
portfolio_std_dev_updated = np.sqrt(
    (weight_xom ** 2 * std_dev_xom ** 2) +
    (weight_dri ** 2 * std_dev_dri ** 2) +
    (2 * weight_xom * weight_dri * std_dev_xom * std_dev_dri * correlation_xom_dri_updated)
)

print(portfolio_std_dev_updated)


9.469444498437767


In [9]:
import statsmodels.api as sm

# Prepare the regression data
X = sm.add_constant(data['Excess_MKT'])  # market excess returns as independent variable
Y_xom = data['Excess_XOM']              # Exxon's excess returns as dependent variable
Y_dri = data['Excess_DRI']              # Darden's excess returns as dependent variable

# Run the regression for Exxon
model_xom = sm.OLS(Y_xom, X).fit()

# Run the regression for Darden
model_dri = sm.OLS(Y_dri, X).fit()

# Get summary of the models
model_xom_summary = model_xom.summary()
model_dri_summary = model_dri.summary()

print(model_xom_summary), (model_dri_summary)


                            OLS Regression Results                            
Dep. Variable:             Excess_XOM   R-squared:                       0.350
Model:                            OLS   Adj. R-squared:                  0.339
Method:                 Least Squares   F-statistic:                     31.25
Date:                Tue, 30 Apr 2024   Prob (F-statistic):           6.39e-07
Time:                        01:55:51   Log-Likelihood:                -210.26
No. Observations:                  60   AIC:                             424.5
Df Residuals:                      58   BIC:                             428.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1632      1.064      0.153      0.8

(None,
 <class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:             Excess_DRI   R-squared:                       0.394
 Model:                            OLS   Adj. R-squared:                  0.383
 Method:                 Least Squares   F-statistic:                     37.68
 Date:                Tue, 30 Apr 2024   Prob (F-statistic):           8.05e-08
 Time:                        01:55:51   Log-Likelihood:                -211.51
 No. Observations:                  60   AIC:                             427.0
 Df Residuals:                      58   BIC:                             431.2
 Df Model:                           1                                         
 Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
 --------------------------------------------------------------

In [10]:
from scipy.optimize import minimize

# Convert the annual target standard deviation to a monthly standard deviation
monthly_target_std_dev = annual_target_std_dev / np.sqrt(12) / 100  # convert percentage to decimal

# Define the function to calculate the portfolio standard deviation
def portfolio_std_dev(weights, std_dev_xom, std_dev_dri, correlation):
    w_xom, w_dri = weights
    w_rf = 1 - w_xom - w_dri
    # Compute the portfolio standard deviation
    return np.sqrt((w_xom * std_dev_xom) ** 2 + (w_dri * std_dev_dri) ** 2 + 
                   2 * w_xom * w_dri * std_dev_xom * std_dev_dri * correlation)

# Define the objective function (negative of the portfolio's return to minimize)
def negative_portfolio_return(weights, avg_ret_xom, avg_ret_dri, rf_rate):
    w_xom, w_dri = weights
    w_rf = 1 - w_xom - w_dri
    return -(w_xom * avg_ret_xom + w_dri * avg_ret_dri + w_rf * rf_rate)

# Constraints (portfolio standard deviation equal to target, sum of weights equal to 1)
constraints = [
    {'type': 'eq', 'fun': lambda weights: portfolio_std_dev(weights, std_dev_xom, std_dev_dri, correlation_xom_dri) - monthly_target_std_dev},
    {'type': 'eq', 'fun': lambda weights: sum(weights) - 1}
]

# Bounds for weights (no short selling, weights between 0 and 1)
bounds = [(0, 1), (0, 1)]

# Initial guess (equally distributed weights)
initial_guess = [0.5, 0.5]

# Run the optimizer
result = minimize(negative_portfolio_return, initial_guess, args=(average_xom, average_dri, average_rf),
                  constraints=constraints, bounds=bounds, method='SLSQP')

print(result.x), (-result.fun)


[0.52905987 0.47094013]


(None, 1.174889225677219)